# 神经机器翻译系统

用TensorFlow搭建seq2seq模型实现了一个简单的神经机器翻译系统，实现英语翻译为法语。Encoder使用双向LSTM。Decoder采用了attention机制。

只使用一张默认计算图，在一个session中进行训练和推断。

详见TensorFlow教程：https://tensorflow.google.cn/tutorials/seq2seq

### 导入包
检查TensorFlow版本和GPU情况

In [1]:
from distutils.version import LooseVersion
import warnings, os
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.7.0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: No GPU found. Please use a GPU to train your neural network.
  if sys.path[0] == '':


### 超参数设置

In [2]:
source_path = 'en-fr/small_vocab_en'
target_path = 'en-fr/small_vocab_fr'
batch_size = 256
num_units = 64
max_gradient_norm = 5.0
learning_rate = 0.001
epoch = 8

### 建立lookup table文件
sos句子开始。eos句子结束。

In [3]:
l=[]
with open(source_path, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        l += line.split()
f.close()
unique_words_src = ['eos'] + list(set(l))

with open('en-fr/words_en', 'w', encoding='utf-8') as f:
    for word in unique_words_src:
        f.write(word + '\n')
f.close()

In [4]:
l=[]
with open(target_path, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        l += line.split()
f.close()
unique_words_tar = ['sos'] + ['eos'] + list(set(l))
print(len(l), len(unique_words_tar))
#print ('rusty' in unique_words_en)

with open('en-fr/words_fr', 'w', encoding='utf-8') as f:
    for word in unique_words_tar:
        f.write(word + '\n')
f.close()

1961295 357


#### 建立lookup table

In [5]:
lookup_en = tf.contrib.lookup.index_table_from_file('en-fr/words_en')
lookup_fr = tf.contrib.lookup.index_table_from_file('en-fr/words_fr')
lookup_translate = tf.contrib.lookup.index_to_string_table_from_file('en-fr/words_fr')
#print (lookup_en)

Instructions for updating:
Use the retry module or similar alternatives.


#### source的末尾标识符，target的开头和末尾标识符

In [6]:
src_eos_id=lookup_en.lookup(tf.constant('eos')) #0 in source vocab
tar_sos_id=lookup_fr.lookup(tf.constant('sos')) #0 in target vocab
tar_eos_id=lookup_fr.lookup(tf.constant('eos')) #1 in target vocab

### 输入文本预处理

#### source dataset
文本转成单词id

In [7]:
source_dataset = tf.data.TextLineDataset(source_path)
source_dataset = source_dataset.map(lambda string: tf.string_split([string]).values)
source_dataset = source_dataset.map(lambda words: (words, tf.size(words)))
source_dataset = source_dataset.map(lambda words, size: (lookup_en.lookup(words), size))

#### target dataset
文本转成单词id。开头加上了一个sos

In [8]:
target_dataset = tf.data.TextLineDataset(target_path)
target_dataset = target_dataset.map(lambda string: tf.string_split([tf.string_join([tf.constant('sos'), string], separator=' ')]).values)
target_dataset = target_dataset.map(lambda words: (words, tf.size(words)))
target_dataset = target_dataset.map(lambda words, size: (lookup_fr.lookup(words), size))

#### 分batch并pad
合并source和target。分成batch。末尾用eos补足到最大长度。无需drop remainder，会自动计算最后一批的样本量。但是后面不能再用batch size。

In [9]:
source_target_dataset = tf.data.Dataset.zip((source_dataset, target_dataset))
'''
batched_dataset = source_target_dataset.apply(tf.contrib.data.padded_batch_and_drop_remainder(
        batch_size,
        padded_shapes=((tf.TensorShape([None]),  # source vectors of unknown size
                        tf.TensorShape([])),     # size(source)
                       (tf.TensorShape([None]),  # target vectors of unknown size
                        tf.TensorShape([]))),    # size(target)
        padding_values=((src_eos_id,  # source vectors padded on the right with src_eos_id
                         0),          # size(source) -- unused
                        (tar_eos_id,  # target vectors padded on the right with tar_eos_id
                         0))))
'''
batched_dataset = source_target_dataset.padded_batch(
        batch_size,
        padded_shapes=((tf.TensorShape([None]),  # source vectors of unknown size
                        tf.TensorShape([])),     # size(source)
                       (tf.TensorShape([None]),  # target vectors of unknown size
                        tf.TensorShape([]))),    # size(target)
        padding_values=((src_eos_id,  # source vectors padded on the right with src_eos_id
                         0),          # size(source) -- unused
                        (tar_eos_id,  # target vectors padded on the right with tar_eos_id
                         0)))         # size(target) -- unused

#print (batched_dataset)

#### 设置iterator

In [10]:
batched_iterator = batched_dataset.make_initializable_iterator()
((source, source_lengths), (target, target_lengths)) = batched_iterator.get_next()

#### Time major 顺序
decoder的输出相比于输入，把开头的sos去掉，末尾补一个eos。
target weight用来做计算loss时的mask

In [11]:
encoder_inputs = tf.transpose(source, [1,0])
decoder_inputs = tf.transpose(target, [1,0])
decoder_outputs = tf.pad(decoder_inputs[1:], tf.constant([[0,1],[0,0]]), constant_values=tar_eos_id)

shape = tf.shape(decoder_outputs)
target_weights = tf.to_double(tf.where(tf.equal(decoder_outputs, tf.fill(shape, tar_eos_id)), tf.zeros(shape), tf.ones(shape)))

#print (encoder_inputs)

### 使用预训练词向量 FastText

In [12]:
embed_file_src = os.path.join('.', 'fasttext', 'wiki-news-300d-1M.vec')
embed_file_tar = os.path.join('.', 'fasttext', 'cc.fr.300.vec')
embed_size = 300

In [13]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index_src = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embed_file_src, encoding = 'utf-8'))
embeddings_index_tar = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embed_file_tar, encoding = 'utf-8'))

#### 建立两种语言的embedding matrix

In [14]:
embedding_matrix_src = np.random.normal(size=(len(unique_words_src), embed_size), scale=0.01)
embedding_matrix_tar = np.random.normal(size=(len(unique_words_tar), embed_size), scale=0.01)

for i, word in enumerate(unique_words_src):
    embedding_vector = embeddings_index_src.get(word)
    if embedding_vector is not None: embedding_matrix_src[i] = embedding_vector
        
for i, word in enumerate(unique_words_tar):
    embedding_vector = embeddings_index_tar.get(word)
    if embedding_vector is not None: embedding_matrix_tar[i] = embedding_vector
        
with tf.variable_scope('embedding'):
    embedding_encoder = tf.Variable(embedding_matrix_src, name='embedding_encoder')
    embedding_decoder = tf.Variable(embedding_matrix_tar, name='embedding_decoder')

### Embedding 层

In [15]:
# Look up embedding:
#   encoder_inputs: [max_time, batch_size]
#   encoder_emb_inp: [max_time, batch_size, embedding_size]
encoder_emb_inp = tf.nn.embedding_lookup(embedding_encoder, encoder_inputs)
decoder_emb_inp = tf.nn.embedding_lookup(embedding_decoder, decoder_inputs)

### Encoder

#### 单向rnn

# Build RNN cell
#with tf.variable_scope('encoder', reuse=True):
encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'encoder_cell')

# Run Dynamic RNN
#   encoder_outputs: [max_time, batch_size, num_units]
#   encoder_state: [batch_size, num_units]
encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, encoder_emb_inp, dtype=tf.float64, sequence_length=source_lengths, time_major=True)

#### 双向rnn

In [16]:
# Construct forward and backward cells
forward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'forward_cell')
backward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'backward_cell')

bi_outputs, encoder_state = tf.nn.bidirectional_dynamic_rnn(
    forward_cell, backward_cell, encoder_emb_inp,dtype=tf.float64,
    sequence_length=source_lengths, time_major=True)
encoder_outputs = tf.concat(bi_outputs, -1)
#encoder_state: LSTM state tuple (cell_state, hidden_state)

### Attention

In [17]:
# attention_states: [batch_size, max_time, num_units] 
# Need to convert encoder hidden states from time major to batch major
attention_states = tf.transpose(encoder_outputs, [1, 0, 2])

In [18]:
# Create an attention mechanism
attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units, attention_states, memory_sequence_length=source_lengths, dtype=tf.float64)

In [19]:
# Build RNN cell and add the attention wrapper 
decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units, name = 'decoder_cell')
decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism, attention_layer_size=num_units)

# set up the initial (zero) state for the attention wrapper. Need to convert encoder state to attention wrapper state
initial_state = decoder_cell.zero_state(dtype=tf.float64, batch_size=tf.shape(encoder_inputs)[1])
initial_state = initial_state.clone(cell_state=encoder_state[0])

### Decoder for training with attention mechanism

In [20]:
# Projection layer on the top
projection_layer = tf.layers.Dense(len(unique_words_tar), use_bias=False, name='projection')

In [21]:
with tf.variable_scope('decoder'):
    # Helper
    helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inp, target_lengths, time_major=True)
    
    # Decoder
    decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state, output_layer=projection_layer)
    
    # Dynamic decoding
    outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=True, impute_finished=True)
    
logits = outputs.rnn_output

### Loss

In [24]:
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=decoder_outputs, logits=logits)
train_loss = (tf.reduce_sum(crossent * target_weights)/ tf.to_double(tf.shape(encoder_inputs)[1]))

In [25]:
# Calculate and clip gradients
params = tf.trainable_variables()
gradients = tf.gradients(train_loss, params)
clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)

In [26]:
# Optimization
optimizer = tf.train.AdamOptimizer(learning_rate)
update_step = optimizer.apply_gradients(zip(clipped_gradients, params))

with tf.Session() as sess: # debug
    sess.run(tf.global_variables_initializer())
    tf.tables_initializer().run()
    sess.run(batched_iterator.initializer)
    inp, lab, log = sess.run([encoder_state, decoder_outputs, logits])
    print (np.shape(inp[1]), np.shape(lab), np.shape(log), encoder_cell.state_size)

    print (inp[1])

### train the model and save the variables

In [27]:
saver = tf.train.Saver(max_to_keep=2)

sess = tf.Session()

sess.run(tf.global_variables_initializer())
tf.tables_initializer().run(session=sess)
#saver.restore(sess, './tmp-model.ckpt-11')
for i in tqdm(range(epoch)):
    sess.run(batched_iterator.initializer)
    n_batch=0
    while True:
        try:
            cost, _ = sess.run([train_loss, update_step])
            n_batch+=1
            print (n_batch)
        except tf.errors.OutOfRangeError:
            print (cost)
            break
    model_path = saver.save(sess, './tmp-model.ckpt', global_step=i+1)
sess.close()

  0%|                                                    | 0/8 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


 12%|█████▏                                   | 1/8 [10:48<1:15:37, 648.17s/it]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


 25%|██████████▎                              | 2/8 [24:31<1:13:33, 735.55s/it]

...

 75%|██████████████████████████████▊          | 6/8 [1:19:56<26:38, 799.48s/it]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


 88%|███████████████████████████████████▉     | 7/8 [1:33:52<13:24, 804.57s/it]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


100%|█████████████████████████████████████████| 8/8 [1:48:06<00:00, 810.76s/it]


### Decoder for inference

In [30]:
with tf.variable_scope('decoder', reuse=True):
    
    # Helper
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding_decoder, tf.fill([tf.shape(encoder_inputs)[1]], tf.to_int32(tar_sos_id)), tf.to_int32(tar_eos_id))

    # Decoder
    decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state, output_layer=projection_layer)

    # Dynamic decoding
    maximum_iterations = tf.round(tf.reduce_max(source_lengths) * 2)
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, maximum_iterations=maximum_iterations, output_time_major=True, impute_finished=True)
    
translation_id = tf.to_int64(outputs.sample_id)
translation = lookup_translate.lookup(translation_id)

### Translate new sentences

In [33]:
# Still use the old session! Do not create a new one and initialize
sess = tf.Session()
tf.tables_initializer().run(session=sess)
saver.restore(sess, model_path)
sess.run(batched_iterator.initializer)
n_batch=0
f = open('en-fr/trans_fr', 'w', encoding='utf-8')
while True:
    try:
        tar_sentences = sess.run(translation)
        n_batch+=1
        tar_sentences = np.transpose(tar_sentences)
        for sentence in tar_sentences:
            for word in sentence:
                if word == 'eos': break
                f.write(word.decode('utf-8') + ' ')
            f.write('\n')
        print(n_batch)
    except tf.errors.OutOfRangeError:
        break
f.close()

# Close the session manually and release resources
sess.close()

INFO:tensorflow:Restoring parameters from ./tmp-model.ckpt-8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262

saver = tf.train.Saver()
with tf.Session() as sess: #debug
    sess.run(tf.global_variables_initializer())
    tf.tables_initializer().run()
    sess.run(batched_iterator.initializer)
    n_batch=0
    ei, di, do = sess.run([encoder_inputs, decoder_inputs, decoder_outputs])
            #print (np.shape(ei), np.shape(di), np.shape(do))
    n_batch+=1
    print(n_batch)
    model_path = saver.save(sess, './tmp-model.ckpt')